In [22]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [23]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Yellowknife,CA,62.46,-114.35,60.80,63,75,11.41,broken clouds,0.0,0
1,1,The Valley,AI,18.22,-63.06,87.80,58,40,18.34,scattered clouds,0.0,0
2,2,Hobart,AU,-42.88,147.33,46.40,70,40,12.75,scattered clouds,0.0,0
3,3,Mantua,IT,45.17,10.78,78.01,63,5,10.00,clear sky,0.0,0
4,4,Sucua,EC,-2.47,-78.17,66.20,93,90,5.82,light rain,0.0,0


In [24]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [25]:
rain_status = input("Do you want it to be raining? (yes/no) ")
snow_status = input("Do you want it to be snowing? (yes/no) ")

Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [26]:
bool(snow_status=='yes')

False

In [27]:
# Filter the dataset to find the cities that fit the temperature criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)                                    
                                      ]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
3,3,Mantua,IT,45.17,10.78,78.01,63,5,10.00,clear sky,0.00,0
5,5,Kaifeng,CN,34.79,114.35,82.40,69,20,6.71,few clouds,0.00,0
6,6,Bethel,US,41.37,-73.41,84.00,34,1,11.41,clear sky,0.00,0
13,13,Kutum,SD,14.20,24.67,78.76,38,93,11.88,overcast clouds,0.00,0
15,15,Northam,GB,51.03,-4.22,75.99,74,100,1.01,light rain,0.26,0
21,21,Benjamin Constant,BR,-4.38,-70.03,84.20,74,78,4.70,moderate rain,1.50,0
22,22,Puerto Escondido,MX,15.85,-97.07,84.20,70,75,12.75,broken clouds,0.00,0
25,25,Atuona,PF,-9.80,-139.03,79.66,74,9,17.83,clear sky,0.00,0
30,30,Banda Aceh,ID,5.56,95.32,81.23,67,100,4.81,overcast clouds,0.00,0
35,35,Pangoa,PE,-12.12,-73.00,84.83,53,78,3.47,broken clouds,0.00,0


In [28]:
# Include rain and snow criteria
if rain_status == 'yes':
    if snow_status== 'yes':
        preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Rain (inches)"]!=0) & \
                                                     (preferred_cities_df["Snow (inches)"]!=0)]
    else:
        preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Rain (inches)"]!=0) & \
                                                     (preferred_cities_df["Snow (inches)"]==0)]
elif rain_status == 'no':
    if snow_status == 'yes':
        preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Rain (inches)"]==0) & \
                                                     (preferred_cities_df["Snow (inches)"]!=0)]
    else:
        preferred_cities_df = preferred_cities_df.loc[(preferred_cities_df["Rain (inches)"]==0) & \
                                                     (preferred_cities_df["Snow (inches)"]==0)]
preferred_cities_df.head(5)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
3,3,Mantua,IT,45.17,10.78,78.01,63,5,10.00,clear sky,0.0,0
5,5,Kaifeng,CN,34.79,114.35,82.40,69,20,6.71,few clouds,0.0,0
6,6,Bethel,US,41.37,-73.41,84.00,34,1,11.41,clear sky,0.0,0
13,13,Kutum,SD,14.20,24.67,78.76,38,93,11.88,overcast clouds,0.0,0
22,22,Puerto Escondido,MX,15.85,-97.07,84.20,70,75,12.75,broken clouds,0.0,0


In [29]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Mantua,IT,78.01,clear sky,45.17,10.78,
5,Kaifeng,CN,82.40,few clouds,34.79,114.35,
6,Bethel,US,84.00,clear sky,41.37,-73.41,
13,Kutum,SD,78.76,overcast clouds,14.20,24.67,
22,Puerto Escondido,MX,84.20,broken clouds,15.85,-97.07,
25,Atuona,PF,79.66,clear sky,-9.80,-139.03,
30,Banda Aceh,ID,81.23,overcast clouds,5.56,95.32,
35,Pangoa,PE,84.83,broken clouds,-12.12,-73.00,
37,San Ramon,US,79.00,clear sky,37.78,-121.98,
40,Abadiania,BR,81.48,clear sky,-16.20,-48.71,


In [30]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [31]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [32]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Mantua,IT,78.01,clear sky,45.17,10.78,Hotel la Favorita
5,Kaifeng,CN,82.40,few clouds,34.79,114.35,Pullman Kaifeng Jianye
6,Bethel,US,84.00,clear sky,41.37,-73.41,Courtyard by Marriott Danbury
13,Kutum,SD,78.76,overcast clouds,14.20,24.67,Dibbo areaمنطقة دبو
22,Puerto Escondido,MX,84.20,broken clouds,15.85,-97.07,Aldea del Bazar Hotel and Spa
25,Atuona,PF,79.66,clear sky,-9.80,-139.03,Villa Enata
30,Banda Aceh,ID,81.23,overcast clouds,5.56,95.32,OYO 884 Rumoh PMI Hotel
35,Pangoa,PE,84.83,broken clouds,-12.12,-73.00,
37,San Ramon,US,79.00,clear sky,37.78,-121.98,San Ramon Marriott
40,Abadiania,BR,81.48,clear sky,-16.20,-48.71,Pousada Lírio D'Água


In [33]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
gmaps.configure(api_key=g_key)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations)

fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [34]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [35]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

In [36]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [37]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))